In [111]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
import os

from src.model import GPTConfig, GPT
import tiktoken

In [113]:
num_return_sequences = 5
max_length = 30

In [115]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

In [128]:
model = GPT.from_pretrained("gpt2")
model.eval()
model.to(device)
print(next(model.parameters()).device)

loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
mps:0


In [125]:
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model,")

tokens = torch.tensor(tokens)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences,1)
tokens = tokens.to(device)
print(tokens.device)

mps:0


In [126]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

x = tokens
while x.size(1) < max_length:
    with torch.no_grad():
        # Sample next token
        logits = model(x)[:,-1,:]

        # Only keep top k tokens
        topk_logits, topk_indices = torch.topk(logits, 50, dim=-1)

        # Compute probs
        topk_probs = F.softmax(topk_logits, dim=-1)

        # Sample best indices
        ix = torch.multinomial(topk_probs, 1)

        # Get generated tokens
        xcol = torch.gather(topk_indices, -1, ix)

        # Concatenate on the last dim
        x = torch.concat((x, xcol), dim=-1)

In [127]:
# Print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(decoded)

Hello, I'm a language model, that's very, where I have, my, "I, I can be all like for me, my
Hello, I'm a language model, and in the game, the games, the are actually a part of the whole, is that.



Hello, I'm a language model, I'm a person, a human being, being a human being a human being, being human being be being
Hello, I'm a language model, and I'm so and I'm so and I'm and I'm I and I am I'm I'm
Hello, I'm a language model, and I'm a natural language language and, I'm, I, I'm a human person. I'm


# Get Shakespear Dataset

In [133]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2025-10-03 08:51:08--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 1115394 (1,1M) [text/plain]
Sauvegarde en : « input.txt »

input.txt           100%[===================>]   1,06M  --.-KB/s    ds 0,06s   

2025-10-03 08:51:09 (17,3 MB/s) — « input.txt » sauvegardé [1115394/1115394]



In [137]:
with open("input.txt", "r", encoding="utf-8") as f:
    text=f.read()
data = text[:1000]
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [142]:
tokens = enc.encode(data)

In [147]:
buf = torch.tensor(tokens[:128+1])
x = buf[:-1].view(8,16)
y = buf[1:].view(8,16)
print(x)
print(y)

tensor([[ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,
          3285,   502,  2740,    13,   198,   198],
        [ 3237,    25,   198,  5248,   461,    11,  2740,    13,   198,   198,
          5962, 22307,    25,   198,  1639,   389],
        [  477, 12939,  2138,   284,  4656,   621,   284,  1145,   680,    30,
           198,   198,  3237,    25,   198,  4965],
        [ 5634,    13, 12939,    13,   198,   198,  5962, 22307,    25,   198,
          5962,    11,   345,   760,   327,  1872],
        [  385,  1526, 28599,   318,  4039,  4472,   284,   262,   661,    13,
           198,   198,  3237,    25,   198,  1135],
        [  760,   470,    11,   356,   760,   470,    13,   198,   198,  5962,
         22307,    25,   198,  5756,   514,  1494],
        [  683,    11,   290,   356,  1183,   423, 11676,   379,   674,   898,
          2756,    13,   198,  3792,   470,   257],
        [15593,    30,   198,   198,  3237,    25,   198,  2949,   517,  3375,
    